In [1]:
from tut import *

Light(children=(Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x00K\x00\x00\x00x\x08\x06\x00\x00\x0…

Switch(value=False, button_style='success', description='Turn Light On')

Thermostat(children=(Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x05\xdc\x00\x00\x04\x04\x08\x02…

first attempt without permissions:
code: 911
message: "(911: couldn\'t find a proof)"



In [2]:
packed = pack_payload(proof2.proofDER, json.dumps({'state': 'on'}))
client.publish("michael/smarthome/light/control", packed)

In [3]:
packed = pack_payload(proof3.proofDER, json.dumps({'hsp': 68}))
client.publish("michael/smarthome/thermostat/control", packed)

michael/smarthome/light/control
got light command

michael/smarthome/thermostat/control
thermostat command {'hsp': 70, 'csp': 78}
michael/smarthome/light/control
got light command

michael/smarthome/thermostat/control
thermostat command {'hsp': 68}
